In [3]:
import pandas as pd 
import os
import pyodbc
import configparser
from Functions.sql_processes import *

In [4]:
# Load Config
config = configparser.ConfigParser()
config.read('Config/config.ini')

conn_string = config['Production']['conn_string']

In [5]:
directory = "./File_Storage/in/Bike_Data"

for filename in os.listdir(directory):
    print(os.path.join(directory, filename))

./File_Storage/in/Bike_Data\JC-202004-citibike-tripdata.csv
./File_Storage/in/Bike_Data\JC-202005-citibike-tripdata.csv
./File_Storage/in/Bike_Data\JC-202006-citibike-tripdata.csv
./File_Storage/in/Bike_Data\JC-202007-citibike-tripdata.csv
./File_Storage/in/Bike_Data\JC-202008-citibike-tripdata.csv
./File_Storage/in/Bike_Data\JC-202009-citibike-tripdata.csv
./File_Storage/in/Bike_Data\JC-202010-citibike-tripdata.csv
./File_Storage/in/Bike_Data\JC-202011-citibike-tripdata.csv
./File_Storage/in/Bike_Data\JC-202101-citibike-tripdata.csv
./File_Storage/in/Bike_Data\JC-202102-citibike-tripdata.csv
./File_Storage/in/Bike_Data\JC-202103-citibike-tripdata.csv
./File_Storage/in/Bike_Data\JC-202104-citibike-tripdata.csv


In [6]:
full_df = pd.DataFrame()
for filename in os.listdir(directory):
    file_path = os.path.join(directory, filename)
    %time df = pd.read_csv(file_path)
    print(filename + " has a total of " + str(df.shape[0]) + " Records")
    full_df = full_df.append(df, ignore_index= True)

Wall time: 323 ms
JC-202004-citibike-tripdata.csv has a total of 9268 Records
Wall time: 81.7 ms
JC-202005-citibike-tripdata.csv has a total of 25077 Records
Wall time: 106 ms
JC-202006-citibike-tripdata.csv has a total of 36921 Records
Wall time: 114 ms
JC-202007-citibike-tripdata.csv has a total of 38680 Records
Wall time: 119 ms
JC-202008-citibike-tripdata.csv has a total of 43267 Records
Wall time: 173 ms
JC-202009-citibike-tripdata.csv has a total of 53833 Records
Wall time: 84 ms
JC-202010-citibike-tripdata.csv has a total of 30086 Records
Wall time: 65 ms
JC-202011-citibike-tripdata.csv has a total of 21275 Records
Wall time: 33.1 ms
JC-202101-citibike-tripdata.csv has a total of 11624 Records
Wall time: 139 ms
JC-202102-citibike-tripdata.csv has a total of 4725 Records
Wall time: 72.4 ms
JC-202103-citibike-tripdata.csv has a total of 16843 Records
Wall time: 70 ms
JC-202104-citibike-tripdata.csv has a total of 23318 Records


In [7]:
full_df.shape[0]

314917

In [8]:
df.isnull().sum()

tripduration               0
starttime                  0
stoptime                   0
start station id           0
start station name         0
start station latitude     0
start station longitude    0
end station id             0
end station name           0
end station latitude       0
end station longitude      0
bikeid                     0
usertype                   0
birth year                 0
gender                     0
dtype: int64

In [9]:
full_df.count()

tripduration               314917
starttime                  314917
stoptime                   314917
start station id           314917
start station name         314917
start station latitude     314917
start station longitude    314917
end station id             314917
end station name           314917
end station latitude       314917
end station longitude      314917
bikeid                     314917
usertype                   314917
birth year                 314917
gender                     314917
dtype: int64

In [10]:
%time write_sql(full_df, 'Bike_Data', conn_string)

Wall time: 2min 4s


In [12]:
%time full_df.to_csv('./File_Storage/out/full_bikedata.csv', index= False)

Wall time: 3.13 s


In [13]:
server = 'localhost'
database = 'FunctionalPy'
filepath = 'C:/Users/JoshuaHigginbotham/source/repos/Functional_Python_for_ETL/File_Storage/out/full_bikedata.csv'
 
# SQL Server Connection
conn = pyodbc.connect(
    'Driver={SQL Server};'
    'Server='+ server + ';'
    'Database=' + database + ';'
    'Trusted_Connection=yes;'
    )
 
bulksql = f"""
    BULK INSERT [dbo].[bike_data_bulk]
    FROM '{filepath}'
    WITH (
        DATAFILETYPE = 'char'
        ,FIELDTERMINATOR = ','
        ,ROWTERMINATOR = '0x0a'
        ,FIRSTROW = 2
        ,KEEPNULLS
        );
"""
 
# Open the connection / cursor
cursor = conn.cursor()
 
# Execute the SQL statement
cursor.execute(bulksql)
cursor.close()